In [1]:
import numpy as np
import torch
from mcspace.model import MCSPACE
from mcspace.trainer import train_model
from mcspace.data_utils import get_data, get_mouse_diet_perturbations_dataset
from mcspace.utils import get_device, pickle_load, pickle_save,\
    estimate_process_variance, MODEL_FILE, DATA_FILE, \
    get_mcspace_cooccur_prob, get_sig_perturbation_diff_subset, \
    get_posterior_summary_data, get_pert_otu_sub, get_min_loss_path, apply_taxonomy_threshold, \
    get_summary_results
from mcspace import utils as ut
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# import mcspace.visualization as vis
from mcspace.dataset import DataSet
import pandas as pd

import ete3
from Bio import SeqIO, Phylo

import mcspace.vis_tools as vis
import matplotlib.colors as mcolors
import networkx as nx
import scipy.cluster.hierarchy as sch

In [3]:
# change font to Arial
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
plt.rcParams['svg.fonttype'] = 'none'

# Paths

In [5]:
pwd

'/home/gary/PROJECTS-wsl/MCSPACE_revisions/MCSPACE/mcspace/MCSPACE_paper/scripts/sensitivity_analysis/helpers/synthetic'

In [9]:
rootpath = Path("../../../../../")
basepath = Path("./")

In [10]:
respath = rootpath / "MCSPACE_paper" / "results" / "analysis" / "Mouse" 
treepath = rootpath / "MCSPACE_paper" / "datasets" / "mouse_experiments" / "output" 
treefile = "newick_tree_query_reads.nhx"

# Load results

In [11]:
thetadf = pd.read_csv(respath / "assemblages.csv")
betadf = pd.read_csv(respath / "assemblage_proportions.csv", index_col=0)
pertsdf = pd.read_csv(respath / "perturbation_bayes_factors.csv", index_col=0)
radf= pd.read_csv(respath / "relative_abundances.csv", index_col=0)

In [12]:
betadf

,Time,Subject,Assemblage,Value,log10Value
0,10,JX07,A1,0.002411,-2.617755
1,10,JX07,A2,0.313011,-0.504440
2,10,JX07,A3,0.012587,-1.900084
3,10,JX07,A4,0.007574,-2.120692
4,10,JX07,A5,0.000553,-3.257598
...,...,...,...,...,...
1318,76,JX10,A59,0.000034,-4.474236
1319,76,JX10,A60,0.034920,-1.456925
1320,76,JX10,A61,0.046733,-1.330378
1321,76,JX10,A62,0.156053,-0.806727


In [13]:
pertsdf

,P1,P2,P3
A1,1.101096,0.688335,0.741492
A2,0.653851,0.395928,1.078150
A3,0.392553,702.266500,3706.187500
A4,7.955726,0.525931,0.614689
A5,8.342922,0.729585,0.557456
...,...,...,...
A59,1.006101,0.425378,0.448774
A60,0.478919,0.603544,0.500608
A61,0.578925,0.606298,2.289281
A62,0.494007,0.684509,0.943481


## note: can have this rely on existing run of mouse data, or can save the results locally so we can extract relevant parameters...